## Feature engineering notebook guide

This notebook builds features and scoring consistent with the EDA stage. It documents what each code block does so readers can follow the pipeline end‑to‑end.

- **Setup and loading**: Load the combined dataset and apply the same cleaning as EDA (drop 0‑minute rows; keep players with ≥200 minutes).
- **Position metrics**: Define per‑position metric sets that mirror the correlation analysis from EDA.
- **Multicollinearity checks**:
  - Compute VIF per position on numeric features.
  - Flag high‑correlation pairs (|r| ≥ 0.8) and save CSVs per position.
- **Feature selection summary**: Document which metrics are retained per position and why.
- **Season comparison tables**: Produce season‑by‑season descriptive tables (seasons as columns, metrics as rows) for 22_23, 23_24, 24_25.
- **Fixed rebalanced scoring**: Compute position‑aware `Rebalanced_Score` with minutes normalization and caps; validate distributions and top performers.
- **Artifacts**: Save VIF tables, correlation pairs, and the final `real_madrid_rebalanced_scores.csv` to the outputs folder.

Outputs: `Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/`.



In [3]:
# ================================
# Setup: load & clean data using EDA rules + output helpers
# ================================
from pathlib import Path
from typing import Optional
import pandas as pd
import numpy as np

# --- Inputs/Outputs ---
INPUT_PATH = Path(r"/Users/home/Capstone/ADS599_Capstone/ADS599_Capstone/Main Notebook/Data Folder/DataCombined/001_real_madrid_all_seasons_combined.csv")
OUTPUT_DIR = Path(r"/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Position metric sets (match your correlation analysis) ---
position_metrics = {
    'Forward':     [' Gls', ' Ast', ' Sh', ' SoT', 'Expected xG', 'Expected npxG', 'Expected xAG', 'Take-Ons Succ', 'Take-Ons Att', 'SCA', 'GCA'],
    'Midfielder':  ['Passes Cmp%', 'KP', ' Tkl', 'SCA', 'GCA', 'Passes PrgP', ' Touches', 'Passes Att', 'Passes Cmp', ' xAG', 'Carries PrgC'],
    'Defender':    [' Tkl', ' Int', ' Blocks', 'Clr', 'Tackles TklW', 'Challenges Tkl%', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Blocks Sh', 'Blocks Pass', 'Tkl+Int'],
    'Goalkeeper':  ['Total Cmp%', 'Err', 'Total TotDist', 'Total PrgDist', 'Long Cmp%', 'Short Cmp%', 'Medium Cmp%', 'Total Cmp', 'Total Att', 'Long Att', 'Short Att']
}

position_mapping = {
    'FW': 'Forward', 'LW': 'Forward', 'RW': 'Forward', 'CF': 'Forward',
    'CM': 'Midfielder', 'DM': 'Midfielder', 'AM': 'Midfielder', 'LM': 'Midfielder', 'RM': 'Midfielder', 'MF': 'Midfielder',
    'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'DF': 'Defender',
    'GK': 'Goalkeeper'
}

# --- Save helpers ---
def save_vif_results(position: str, vif_results: pd.DataFrame) -> None:
    """Save a VIF table for a position to the outputs folder."""
    out_path = OUTPUT_DIR / f"vif_{position.lower()}.csv"
    pd.DataFrame(vif_results).to_csv(out_path, index=False)
    print(f"✓ VIF results saved → {out_path}")

def save_problematic_pairs(position: str, problematic_pairs: pd.DataFrame) -> None:
    """Save high-correlation pairs (|r| threshold breaches) for a position."""
    out_path = OUTPUT_DIR / f"corr_pairs_{position.lower()}.csv"
    pd.DataFrame(problematic_pairs, columns=["Metric_1","Metric_2","Correlation","Severity"]).to_csv(out_path, index=False)
    print(f"✓ High correlation pairs saved → {out_path}")

# --- EDA-consistent loading/cleaning ---
def load_and_clean_data(path: Path = INPUT_PATH, minutes_threshold: int = 200) -> pd.DataFrame:
    """
    Replicates EDA cleaning:
      1) drop zero-minute rows
      2) keep players with total minutes ≥ threshold (across the dataset)
    """
    if not path.exists():
        raise FileNotFoundError(f"Input file not found: {path}")

    df = pd.read_csv(path)

    # Standardize minutes column
    minute_col: Optional[str] = None
    for cand in ["Minutes", "Min", "mins", "minute", "minutes"]:
        if cand in df.columns:
            minute_col = cand
            break

    if minute_col is None:
        print("⚠️ No minutes column found (Minutes/Min). Returning original dataframe.")
        return df

    # Ensure numeric minutes (coerce bad values to NaN, then drop)
    df[minute_col] = pd.to_numeric(df[minute_col], errors="coerce")

    # 1) Remove zero/NaN-minute rows
    before_rows = len(df)
    df = df[df[minute_col].fillna(0) > 0].copy()
    print(f"Removed zero/NaN-minute rows: {before_rows - len(df)}")

    # 2) Keep players with total minutes ≥ threshold
    if "Player" not in df.columns:
        print("⚠️ No 'Player' column found. Skipping per-player minute aggregation filter.")
        return df

    player_minutes = df.groupby("Player", dropna=False)[minute_col].sum()
    players_to_keep = player_minutes[player_minutes >= minutes_threshold].index

    before_players = df["Player"].nunique(dropna=True)
    df = df[df["Player"].isin(players_to_keep)].copy()
    after_players = df["Player"].nunique(dropna=True)
    print(f"Players before/after minutes filter (≥{minutes_threshold}): {before_players} → {after_players}")

    return df

# --- Execute load/clean and expose `df` for downstream cells ---
df = load_and_clean_data(INPUT_PATH, minutes_threshold=200)
print(f"Cleaned dataset shape: {df.shape}")

Removed zero/NaN-minute rows: 8
Players before/after minutes filter (≥200): 82 → 60
Cleaned dataset shape: (7140, 77)


In [4]:
# Multicollinearity and VIF analysis per position with CSV outputs
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

POSITIONS_TO_ANALYZE = ['Forward', 'Midfielder', 'Defender', 'Goalkeeper']

# Utility: map dataset `Pos` abbreviations to a target position label

def _mask_for_position(df: pd.DataFrame, position_label: str) -> pd.Series:
    if 'Pos' not in df.columns:
        return pd.Series([True] * len(df), index=df.index)
    abbrs = [abbr for abbr, full in POSITION_MAPPING.items() if full == position_label]
    mask = df['Pos'].isin(abbrs)
    # include multi-position strings (e.g., "FW,RM")
    for abbr in abbrs:
        mask = mask | df['Pos'].astype(str).str.contains(abbr, case=False, na=False)
    return mask


def _select_numeric_features(df: pd.DataFrame) -> pd.DataFrame:
    numeric_df = df.select_dtypes(include=[np.number]).copy()
    # Drop identifier-like or leakage columns if present
    drop_like = { 'Season' }
    cols = [c for c in numeric_df.columns if c not in drop_like]
    numeric_df = numeric_df[cols]
    # Remove columns with all NaNs or zero variance
    numeric_df = numeric_df.dropna(axis=1, how='all')
    nunique = numeric_df.nunique(dropna=True)
    constant_cols = nunique[nunique <= 1].index.tolist()
    if constant_cols:
        numeric_df = numeric_df.drop(columns=constant_cols, errors='ignore')
    # Drop rows with any NaNs for VIF stability
    numeric_df = numeric_df.dropna(axis=0, how='any')
    return numeric_df


def _compute_vif(df_numeric: pd.DataFrame) -> pd.DataFrame:
    if df_numeric.shape[1] < 2 or df_numeric.shape[0] < 5:
        return pd.DataFrame(columns=['Metric', 'VIF', 'Status'])
    X = df_numeric.values
    columns = df_numeric.columns.tolist()
    vif_rows = []
    for i, col in enumerate(columns):
        try:
            vif_value = variance_inflation_factor(X, i)
        except Exception:
            vif_value = np.nan
        status = 'OK'
        if pd.notna(vif_value):
            if vif_value > 10:
                status = 'Severe (>10)'
            elif vif_value > 5:
                status = 'Moderate (5-10)'
        vif_rows.append({'Metric': col, 'VIF': float(vif_value) if pd.notna(vif_value) else np.nan, 'Status': status})
    return pd.DataFrame(vif_rows)


def _high_correlation_pairs(df_numeric: pd.DataFrame, threshold: float = 0.8) -> pd.DataFrame:
    if df_numeric.shape[1] < 2:
        return pd.DataFrame(columns=["Metric_1","Metric_2","Correlation","Severity"])
    corr = df_numeric.corr().stack().reset_index()
    corr.columns = ['Metric_1', 'Metric_2', 'Correlation']
    corr = corr[corr['Metric_1'] < corr['Metric_2']]
    corr['abs_r'] = corr['Correlation'].abs()
    res = corr[corr['abs_r'] >= threshold].copy()
    if res.empty:
        return pd.DataFrame(columns=["Metric_1","Metric_2","Correlation","Severity"])
    res['Severity'] = np.where(res['abs_r'] >= 0.9, 'Severe (>=0.9)', 'High (0.8-0.9)')
    return res[['Metric_1','Metric_2','Correlation','Severity']].sort_values(by='Correlation', ascending=False)


def test_multicollinearity_by_position_with_outputs(df_full: pd.DataFrame) -> None:
    print("MULTICOLLINEARITY TEST BY POSITION (with outputs)")
    for position in POSITIONS_TO_ANALYZE:
        print("\n" + "-"*70)
        print(f"{position.upper()} MULTICOLLINEARITY TEST")
        mask = _mask_for_position(df_full, position)
        df_pos = df_full.loc[mask].copy()
        if df_pos.empty:
            print("No data for this position; skipping.")
            continue
        df_numeric = _select_numeric_features(df_pos)
        if df_numeric.empty:
            print("No numeric features after cleaning; skipping.")
            continue
        vif_df = _compute_vif(df_numeric)
        if not vif_df.empty:
            print("VIF Results (top 15 by value):")
            display_cols = ['Metric','VIF','Status']
            print(vif_df.sort_values('VIF', ascending=False).head(15)[display_cols].to_string(index=False))
            save_vif_results(position, vif_df)
        else:
            print("VIF not computed (insufficient features/rows).")
        corr_pairs = _high_correlation_pairs(df_numeric)
        if not corr_pairs.empty:
            print("\nHighly correlated pairs (|r| >= 0.8):")
            print(corr_pairs.to_string(index=False))
            save_problematic_pairs(position, corr_pairs)
        else:
            print("- No severe multicollinearity detected (all |r| < 0.8)")

# Run
try:
    test_multicollinearity_by_position_with_outputs(df)
except NameError:
    # if df not defined for some reason, attempt to load/clean again
    df = load_and_clean_data(INPUT_PATH)
    test_multicollinearity_by_position_with_outputs(df)



MULTICOLLINEARITY TEST BY POSITION (with outputs)

----------------------------------------------------------------------
FORWARD MULTICOLLINEARITY TEST
Removed zero/NaN-minute rows: 8
Players before/after minutes filter (≥200): 82 → 60
MULTICOLLINEARITY TEST BY POSITION (with outputs)

----------------------------------------------------------------------
FORWARD MULTICOLLINEARITY TEST


NameError: name 'POSITION_MAPPING' is not defined

In [6]:
# Setup: load and clean data using EDA rules, and define output helpers
import pandas as pd
import numpy as np
from pathlib import Path

# Input dataset from EDA stage
INPUT_PATH = "/Users/home/Capstone/ADS599_Capstone/ADS599_Capstone/Main Notebook/Data Folder/DataCombined/001_real_madrid_all_seasons_combined.csv"

# Outputs directory for feature engineering artifacts
OUTPUT_DIR = "/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Position mapping used in EDA
POSITION_MAPPING = {
    'FW': 'Forward', 'ST': 'Forward', 'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward',
    'AM': 'Midfielder', 'CM': 'Midfielder', 'DM': 'Midfielder', 'RM': 'Midfielder', 'LM': 'Midfielder',
    'LB': 'Defender', 'CB': 'Defender', 'RB': 'Defender', 'LWB': 'Defender', 'RWB': 'Defender',
    'GK': 'Goalkeeper'
}

# Save helpers

def save_vif_results(position: str, vif_results: pd.DataFrame) -> None:
    out_path = Path(OUTPUT_DIR) / f"vif_{position.lower()}.csv"
    pd.DataFrame(vif_results).to_csv(out_path, index=False)
    print(f"✓ VIF results saved → {out_path}")


def save_problematic_pairs(position: str, problematic_pairs: pd.DataFrame) -> None:
    out_path = Path(OUTPUT_DIR) / f"corr_pairs_{position.lower()}.csv"
    pd.DataFrame(problematic_pairs, columns=["Metric_1","Metric_2","Correlation","Severity"]).to_csv(out_path, index=False)
    print(f"✓ High correlation pairs saved → {out_path}")


def load_and_clean_data(path: str = INPUT_PATH, minutes_threshold: int = 200) -> pd.DataFrame:
    """Replicates EDA cleaning: remove zero-minute rows and players with < threshold total minutes."""
    df = pd.read_csv(path)

    # Standardize minute column reference
    minute_col = None
    if 'Minutes' in df.columns:
        minute_col = 'Minutes'
    elif 'Min' in df.columns:
        minute_col = 'Min'

    if minute_col is None:
        print("No 'Minutes' or 'Min' column found - returning original dataframe")
        return df

    # Remove zero-minute appearances
    before_rows = len(df)
    df = df[df[minute_col] > 0].copy()
    print(f"Removed zero-minute rows: {before_rows - len(df)}")

    # Keep players with >= threshold total minutes across entire dataset
    player_minutes = df.groupby('Player')[minute_col].sum()
    players_to_keep = player_minutes[player_minutes >= minutes_threshold].index
    before_players = df['Player'].nunique()
    df = df[df['Player'].isin(players_to_keep)].copy()
    after_players = df['Player'].nunique()
    print(f"Players before/after minutes filter ({minutes_threshold}): {before_players} → {after_players}")

    return df

# Load and clean, then expose as `df` to keep downstream cells working
_df_cleaned = load_and_clean_data(INPUT_PATH)
df = _df_cleaned.copy()
print(f"Cleaned dataset shape: {df.shape}")


Removed zero-minute rows: 8
Players before/after minutes filter (200): 82 → 60
Cleaned dataset shape: (7140, 77)


## 1. Import Libraries and Load Data

In [7]:
# Multicollinearity Test by Position
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

def test_multicollinearity_by_position(df):
    """
    Test multicollinearity for each position and show VIF values
    """
    
    print("MULTICOLLINEARITY TEST BY POSITION")
    print("="*50)
    
    # YOUR EXACT position metrics from correlation analysis
    position_metrics = {
        'Forward': [' Gls', ' Ast', ' Sh', ' SoT', 'Expected xG', 'Expected npxG', 'Expected xAG', 'Take-Ons Succ', 'Take-Ons Att', 'SCA', 'GCA'],
        'Midfielder': ['Passes Cmp%', 'KP', ' Tkl', 'SCA', 'GCA', 'Passes PrgP', ' Touches', 'Passes Att', 'Passes Cmp', ' xAG', 'Carries PrgC'],
        'Defender': [' Tkl', ' Int', ' Blocks', 'Clr', 'Tackles TklW', 'Challenges Tkl%', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Blocks Sh', 'Blocks Pass', 'Tkl+Int'],
        'Goalkeeper': ['Total Cmp%', 'Err', 'Total TotDist', 'Total PrgDist', 'Long Cmp%', 'Short Cmp%', 'Medium Cmp%', 'Total Cmp', 'Total Att', 'Long Att', 'Short Att']
    }
    
    position_mapping = {
        'FW': 'Forward', 'LW': 'Forward', 'RW': 'Forward', 'CF': 'Forward',
        'CM': 'Midfielder', 'DM': 'Midfielder', 'AM': 'Midfielder', 'LM': 'Midfielder', 'RM': 'Midfielder',
        'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'DF': 'Defender'
    }
    
    for position, metrics in position_metrics.items():
        print(f"\n{position.upper()} MULTICOLLINEARITY TEST")
        print("-" * 40)
        
        # Check which metrics are actually in the dataset
        available_metrics = []
        missing_metrics = []
        
        for metric in metrics:
            if metric in df.columns:
                available_metrics.append(metric)
            else:
                missing_metrics.append(metric)
        
        print(f"Total metrics defined: {len(metrics)}")
        print(f"Available metrics: {len(available_metrics)}")
        print(f"Missing metrics: {len(missing_metrics)}")
        
        if missing_metrics:
            print(f"MISSING: {missing_metrics}")
        
        print(f"AVAILABLE: {available_metrics}")
        
        # Filter data for this position
        if position == 'Forward':
            pos_data = df[df['Pos'].str.contains('FW|LW|RW|CF', case=False, na=False)]
        elif position == 'Midfielder':
            pos_data = df[df['Pos'].str.contains('CM|DM|AM|LM|RM', case=False, na=False)]
        elif position == 'Defender':
            pos_data = df[df['Pos'].str.contains('CB|LB|RB|DF', case=False, na=False)]
        elif position == 'Goalkeeper':
            pos_data = df[df['Pos'].str.contains('GK', case=False, na=False)]
        
        if len(available_metrics) < 2:
            print(f"Insufficient available metrics for {position}")
            continue
            
        # Get clean data (no missing values)
        clean_data = pos_data[available_metrics].dropna()
        
        if len(clean_data) < 10:
            print(f"Insufficient data for {position}")
            continue
            
        print(f"Sample size: {len(clean_data)}")
        print(f"Metrics tested: {available_metrics}")
        
        # Calculate correlation matrix
        corr_matrix = clean_data.corr()
        
        # Calculate VIF for each variable
        print(f"\nVIF Results:")
        print(f"{'Metric':<20} {'VIF':<8} {'Status'}")
        print("-" * 40)
        
        vif_results = []
        try:
            for i, metric in enumerate(available_metrics):
                # Calculate VIF
                vif = variance_inflation_factor(clean_data.values, i)
                
                # Determine status
                if vif > 10:
                    status = "SEVERE - Remove"
                elif vif > 5:
                    status = "MODERATE - Consider"
                else:
                    status = "OK"
                
                vif_results.append({'Metric': metric, 'VIF': vif, 'Status': status})
                print(f"{metric:<20} {vif:<8.2f} {status}")
                
        except:
            print("VIF calculation failed - using correlation instead")
            
            # Alternative: High correlation pairs
            print(f"\nHigh Correlation Pairs (r > 0.8):")
            for i in range(len(available_metrics)):
                for j in range(i+1, len(available_metrics)):
                    corr_val = corr_matrix.iloc[i, j]
                    if abs(corr_val) > 0.8:
                        metric1 = available_metrics[i]
                        metric2 = available_metrics[j]
                        print(f"{metric1} ↔ {metric2}: r = {corr_val:.3f} (REMOVE ONE)")
        
        # Dynamic recommendation based on actual VIF/correlation results
        print(f"\nRECOMMENDATION FOR {position.upper()}:")
        
        # Find problematic pairs from correlation matrix
        problematic_pairs = []
        for i in range(len(available_metrics)):
            for j in range(i+1, len(available_metrics)):
                if i < len(corr_matrix.columns) and j < len(corr_matrix.columns):
                    corr_val = corr_matrix.iloc[i, j]
                    if abs(corr_val) > 0.8:
                        metric1 = available_metrics[i]
                        metric2 = available_metrics[j]
                        severity = "SEVERE" if abs(corr_val) > 0.9 else "MODERATE"
                        problematic_pairs.append((metric1, metric2, corr_val, severity))
                        print(f"- PROBLEM: {metric1} ↔ {metric2} (r = {corr_val:.3f}) - {severity}")
        
        if not problematic_pairs:
            print("- No severe multicollinearity detected (all r < 0.8)")
        else:
            print("- Consider removing one variable from each problematic pair")
        
        print("-" * 50)

# Run the test
test_multicollinearity_by_position(df)

# Simple correlation check for selected metrics
print("\n" + "="*60)
print("FINAL RECOMMENDED METRICS - CORRELATION CHECK")
print("="*60)

recommended_metrics = {
    'Forward': 'Expected xG',
    'Midfielder': 'Passes Cmp', 
    'Defender': 'Tackles TklW'
}

print("Checking correlations between recommended metrics:")
for pos, metric in recommended_metrics.items():
    if metric in df.columns:
        # Check correlation with other recommended metrics
        for other_pos, other_metric in recommended_metrics.items():
            if pos != other_pos and other_metric in df.columns:
                corr = df[metric].corr(df[other_metric])
                print(f"{pos} ({metric}) ↔ {other_pos} ({other_metric}): r = {corr:.3f}")

print(f"\nFinal recommendation: Use these 3 metrics to predict Team_xG")
for pos, metric in recommended_metrics.items():
    print(f"- {pos}: {metric}")
print("These metrics have low cross-correlation and capture unique position functions.")

MULTICOLLINEARITY TEST BY POSITION

FORWARD MULTICOLLINEARITY TEST
----------------------------------------
Total metrics defined: 11
Available metrics: 11
Missing metrics: 0
AVAILABLE: [' Gls', ' Ast', ' Sh', ' SoT', 'Expected xG', 'Expected npxG', 'Expected xAG', 'Take-Ons Succ', 'Take-Ons Att', 'SCA', 'GCA']
Sample size: 598
Metrics tested: [' Gls', ' Ast', ' Sh', ' SoT', 'Expected xG', 'Expected npxG', 'Expected xAG', 'Take-Ons Succ', 'Take-Ons Att', 'SCA', 'GCA']

VIF Results:
Metric               VIF      Status
----------------------------------------
 Gls                 2.99     OK
 Ast                 2.54     OK
 Sh                  7.20     MODERATE - Consider
 SoT                 5.53     MODERATE - Consider
Expected xG          12.30    SEVERE - Remove
Expected npxG        12.73    SEVERE - Remove
Expected xAG         2.58     OK
Take-Ons Succ        6.26     MODERATE - Consider
Take-Ons Att         6.64     MODERATE - Consider
SCA                  5.74     MODERATE - Con

/opt/miniconda3/envs/ads509/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


## 2.3. Feature Engineering and Feature selection

Based on the result for collinearity  we slelect the following:
From SEVERE Pairs - Keep One:
FORWARDS:

Expected xG ↔ Expected npxG (r = 0.913): KEEP Expected xG (more standard metric)
Take-Ons Succ ↔ Take-Ons Att (r = 0.851): KEEP Take-Ons Succ (outcome vs attempt)

MIDFIELDERS:

Touches ↔ Passes Att ↔ Passes Cmp (all r > 0.98): KEEP Passes Cmp (most meaningful outcome)

DEFENDERS:

Tkl ↔ Tackles TklW (r = 0.840): KEEP Tackles TklW (successful tackles vs attempts)
Tkl ↔ Tkl+Int (r = 0.812): KEEP Tkl+Int (combined defensive actions)

GOALKEEPERS:

Total TotDist ↔ Total PrgDist (r = 0.893): KEEP Total PrgDist (progressive passing)
Total Cmp ↔ Total Att (r = 0.878): KEEP Total Cmp (successful passes)

REVISED FINAL LIST (OK + MODERATE + One from SEVERE pairs):
FORWARDS: Gls, Ast, SoT, Expected xG, Expected xAG, Take-Ons Succ
MIDFIELDERS: Passes Cmp%, KP, Tkl, Carries PrgC, Passes PrgP, Touches
DEFENDERS: Int, Blocks, Clr, Tackles TklW, Tackles Def 3rd, Tackles Mid 3rd, Blocks Sh, Blocks Pass
GOALKEEPERS: Total Cmp%, Err, Total PrgDist, Short Cmp%, Medium Cmp%, Total Cmp,  Short Att

In [8]:
## 2.4 Descriptive Statistics
# Season-by-Season Descriptive Statistics (After accounting for multicolinearity)
# SEASONS AS COLUMNS, METRICS AS ROWS

def create_season_comparison_tables(df):
    """
    Create tables with seasons as columns (like Ford/Chevy/Ram), metrics as rows
    """
    
    # Position metrics (your final list)
    position_metrics = {
        'Forward': ['Gls', 'Ast', 'SoT', 'Expected xG', 'Expected xAG', 'Take-Ons Succ'],
        'Midfielder': ['Passes Cmp%', 'KP', 'Tkl', 'Carries PrgC', 'Passes PrgP', 'Touches'],
        'Defender': ['Int', 'Blocks', 'Clr', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Blocks Sh', 'Blocks Pass'],
        'Goalkeeper': ['Total Cmp%', 'Err', 'Total PrgDist', 'Short Cmp%', 'Medium Cmp%', 'Total Cmp', 'Short Att']
    }
    
    # Position mapping
    position_mapping = {
        'Forward': ['FW', 'LW', 'RW', 'CF'],
        'Midfielder': ['CM', 'DM', 'AM', 'LM', 'RM'],
        'Defender': ['CB', 'LB', 'RB', 'DF'],
        'Goalkeeper': ['GK']
    }
    
    # Get unique seasons and sort them
    seasons = sorted(df['Season'].unique())
    print(f"Available seasons: {seasons}")
    
    for position, metrics in position_metrics.items():
        
        # Filter data for this position
        pos_codes = position_mapping[position]
        pos_pattern = '|'.join(pos_codes)
        pos_data = df[df['Pos'].str.contains(pos_pattern, case=False, na=False)]
        
        if len(pos_data) == 0:
            continue
            
        # Get available metrics (handle space prefix)
        available_metrics = []
        for m in metrics:
            if m in df.columns:
                available_metrics.append(m)
            elif f' {m}' in df.columns:
                available_metrics.append(f' {m}')
        
        if len(available_metrics) == 0:
            continue
            
        print(f"\nTable {list(position_metrics.keys()).index(position) + 2}. Descriptive Statistics of {position} Performance by Season")
        print("=" * 80)
        print()
        
        # Calculate season-specific data
        season_data = {}
        season_stats = {}
        
        for season in seasons:
            season_pos_data = pos_data[pos_data['Season'] == season]
            if len(season_pos_data) > 0:
                season_data[season] = season_pos_data
                # Calculate total observations and unique players
                total_obs = len(season_pos_data)
                unique_players = season_pos_data['Player'].nunique() if 'Player' in season_pos_data.columns else 'N/A'
                season_stats[season] = {'total_obs': total_obs, 'unique_players': unique_players}
        
        # Use specific seasons: 2022-23, 2023-24, 2024-25
        target_seasons = ['22_23', '23_24', '24_25']
        
        # Filter to only include target seasons
        seasons_to_show = [season for season in target_seasons if season in season_data.keys()]
        
        if len(seasons_to_show) == 0:
            print(f"No data found for target seasons {target_seasons} in {position}")
            continue
        
        # Header with seasons as columns
        header = f"{'Metric':<25}"
        for season in seasons_to_show:
            header += f"{season:<12}"
        print(header)
        print("-" * (25 + len(seasons_to_show) * 12))
        
        # Total observations row
        obs_row = f"{'Total observations':<25}"
        for season in seasons_to_show:
            if season in season_stats:
                obs_row += f"{season_stats[season]['total_obs']:<12,}"
            else:
                obs_row += f"{'--':<12}"
        print(obs_row)
        
        # Unique players row
        players_row = f"{'Unique players':<25}"
        for season in seasons_to_show:
            if season in season_stats:
                players_row += f"{season_stats[season]['unique_players']:<12}"
            else:
                players_row += f"{'--':<12}"
        print(players_row)
        print()
        
        # Process each metric (limit to 4-5 key metrics for readability)
        key_metrics = available_metrics[:5]
        
        for metric in key_metrics:
            # Clean metric name for display
            display_metric = metric.strip().replace('Expected ', 'Exp').replace('Tackles ', 'Tkl').replace('Take-Ons ', 'TO')
            
            print(f"{display_metric}")
            
            # Calculate statistics for each season
            metric_stats = {}
            for season in seasons_to_show:
                if season in season_data and metric in season_data[season].columns:
                    clean_data = season_data[season][metric].dropna()
                    if len(clean_data) > 0:
                        metric_stats[season] = clean_data.describe()
            
            # Display statistics (Ford/Chevy/Ram style)
            stat_labels = ['Mean', 'SD', 'Minimum', '25th percentile', 'Median', '75th percentile', 'Maximum']
            stat_keys = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
            
            for label, key in zip(stat_labels, stat_keys):
                row = f"  {label:<23}"
                for season in seasons_to_show:
                    if season in metric_stats:
                        if key in ['min', 'max']:
                            row += f"{metric_stats[season][key]:<12.0f}"
                        else:
                            row += f"{metric_stats[season][key]:<12.2f}"
                    else:
                        row += f"{'--':<12}"
                print(row)
            print()
        
        print("-" * (25 + len(seasons_to_show) * 12))
        print()
        
        # Note section (APA style)
        print(f"Note. Performance statistics for {position.lower()} players across {len(seasons_to_show)} seasons.")
        print(f"Metrics represent core {position.lower()} performance indicators without multicollinearity.")
        total_all_seasons = sum([season_stats[s]['total_obs'] for s in seasons_to_show if s in season_stats])
        unique_all_seasons = len(set([player for s in seasons_to_show if s in season_data 
                                    for player in season_data[s]['Player'].unique() if 'Player' in season_data[s].columns]))
        print(f"Combined sample: {total_all_seasons:,} observations from {unique_all_seasons} unique players.")
        print()

# Run the analysis
create_season_comparison_tables(df)

Available seasons: ['15_16', '16_17', '17_18', '18_19', '19_20', '20_21', '21_22', '22_23', '23_24', '24_25']

Table 2. Descriptive Statistics of Forward Performance by Season

Metric                   22_23       23_24       24_25       
-------------------------------------------------------------
Total observations       215         199         214         
Unique players           11          12          12          

Gls
  Mean                   0.34        0.36        0.37        
  SD                     0.59        0.61        0.68        
  Minimum                0           0           0           
  25th percentile        0.00        0.00        0.00        
  Median                 0.00        0.00        0.00        
  75th percentile        1.00        1.00        1.00        
  Maximum                3           2           3           

Ast
  Mean                   0.20        0.16        0.14        
  SD                     0.46        0.45        0.37        
  Minim

Season-by-Season Performance Analysis (2022-25)
FORWARDS (Table 2) - Trending Analysis:
IMPROVING TRENDS:

Goal scoring increasing: 0.34 → 0.36 → 0.37 goals per match (steady improvement)
Shot quality rising: Shots on target increased from 0.91 to 0.98
Expected Goals stable: xG around 0.31-0.35 range (consistent threat creation)

DECLINING TRENDS:

Creativity dropping: Assists fell from 0.20 → 0.16 → 0.14 (concerning trend)
Expected Assists down: xAG declined from 0.19 → 0.12 → 0.16 (less playmaking)

KEY INSIGHT: Forwards becoming more selfish but more clinical - scoring more but creating less for teammates.

MIDFIELDERS (Table 3) - Stability with Concerns:
CONSISTENT PERFORMANCE:

Passing accuracy stable: ~87-89% across all seasons (reliable ball retention)
Key passes steady: 1.03 → 1.21 → 1.05 (consistent creativity)
Defensive work increasing: Tackles rose from 1.30 → 1.49 (more defensive responsibility)

⚠️ SLIGHT DECLINE:

Progressive play dropping: Progressive passes fell from 5.32 → 4.82
Ball carrying down: Progressive carries decreased from 1.81 → 1.57

 KEY INSIGHT: Midfielders maintaining core functions but becoming less adventurous in attack.

DEFENDERS (Table 4) - Improving Defensive Intensity:
POSITIVE TRENDS:

More active defending: Clearances increased dramatically (1.77 → 2.32)
Better positioning: Interceptions improved (0.73 → 0.79)
Tackle success up: Successful tackles rose from 0.74 → 0.87

 KEY INSIGHT: Defense becoming more proactive and aggressive - suggests team facing more pressure but handling it better.

GOALKEEPERS (Table 5) - Distribution Concerns:
⚠️ DECLINING TRENDS:

Passing accuracy dropping: Total completion fell from 86.25% → 83.39%
Shorter distribution: Progressive distance decreased (455m → 400m)
More conservative: Playing safer, shorter passes

 POSITIVE:

Error-free: Extremely low error rates (0.02-0.04 per game)
Short passing excellent: 99%+ accuracy on short passes

 KEY INSIGHT: Goalkeepers playing more conservatively, possibly due to tactical changes or pressure.

OVERALL TEAM EVOLUTION (2022-25):

Tactical Shift: From creative to pragmatic - forwards scoring more but assisting less
Defensive Improvement: More active defending suggests better organization
Conservative Approach: Reduced progressive play from midfield and goalkeepers
Efficiency Focus: Better conversion rates but less risk-taking in final third

SUMMARY: Real Madrid evolved from a more creative, risk-taking team to a more efficient, defensively solid unit that relies on clinical finishing rather than elaborate build-up play.

## 2.4 Main metrics for each position


Unique Overall Performance Metrics by Position:
🔥 FORWARDS:
Primary Metric: Expected xG

Why: Best predictor of attacking threat and quality of chances created
Contribution: Measures shooting ability and positioning in dangerous areas
Range: 0.00-2.70 per match

🎯 MIDFIELDERS:
Primary Metric: KP (Key Passes)

Why: Captures creativity and chance creation ability
Contribution: Measures playmaking and ability to unlock defenses
Range: 0-9 per match

🛡️ DEFENDERS:
Primary Metric: Tackles TklW (Successful Tackles)

Why: Best indicator of defensive effectiveness and ball-winning ability
Contribution: Measures active defending and disrupting opponent attacks
Range: 0-6 per match

🥅 GOALKEEPERS:
Primary Metric: Total Cmp% (Distribution Accuracy)

Why: Modern goalkeeper's most important contribution beyond shot-stopping
Contribution: Measures ability to start attacks and maintain possession
Range: 44-100% completion rate

Why These Metrics Are Unique:

Position-Specific: Each captures the core function of that position
Low Cross-Correlation: These metrics don't overlap between positions
Performance Predictive: Best indicators of individual contribution to team success
Multicollinearity-Free: Selected after removing redundant variables

For Team xG Correlation:
These metrics would be the independent variables to predict Team xG (dependent variable), showing how individual position performance contributes to overall team attacking output.

## 3 Four with recalculation of weights


In [9]:
# === FIXED PERFORMANCE SCORING SYSTEM (robust paths + minutes + Series-safe math) ===
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("=== FIXED PERFORMANCE SCORING SYSTEM ===")

# --- 0) Helpers ---------------------------------------------------------------
def pick_col(cols, *candidates):
    """Return the first column name that exists in df.columns from the list of candidates."""
    for c in candidates:
        if c in cols:
            return c
    return None

# 1) Ensure Position_Group exists (skip gracefully if Pos missing)
if 'Pos' in df.columns:
    def categorize_position(pos):
        if pd.isna(pos):
            return None
        s = str(pos).upper()
        if 'GK' in s: return 'Goalkeeper'
        if any(x in s for x in ['FW', 'CF', 'ST', 'LW', 'RW']): return 'Forward'
        if any(x in s for x in ['MF', 'CM', 'DM', 'AM', 'LM', 'RM']): return 'Midfield'
        if any(x in s for x in ['DF', 'CB', 'LB', 'RB', 'WB', 'SW']): return 'Defense'
        return 'Midfield'
    df['Position_Group'] = df['Pos'].apply(categorize_position)
else:
    if 'Position_Group' not in df.columns:
        df['Position_Group'] = np.nan
        print("⚠️ No 'Pos' column found. Position_Group set to NaN.")

print("Dataset shape:", df.shape)
print("Available columns:", list(df.columns))

# --- minutes column detection
MIN_COL = pick_col(df.columns, 'Min', 'Minutes')
if MIN_COL is None:
    raise KeyError("No minutes column found (expected 'Min' or 'Minutes').")

# ==========================================
# FIXED REBALANCED SCORING SYSTEM
# ==========================================
def calculate_rebalanced_scores_fixed(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Rebalanced scoring using only columns that exist. All math done with pandas Series
    so assignments align back to df.
    """
    df = df_in.copy()
    df['Rebalanced_Score'] = 0.0  # base

    available_cols = set(df.columns)

    # ---------------- Goalkeepers ----------------
    gk_mask = df['Position_Group'].eq('Goalkeeper')
    if gk_mask.any():
        print(f"\nRebalancing Goalkeepers ({int(gk_mask.sum())} rows)...")
        gk = df.loc[gk_mask].copy()
        # Start from base 15 as a Series
        gk_score = pd.Series(15.0, index=gk.index)

        # Pass completion %
        cmp_col = pick_col(available_cols, 'Total Cmp%')
        if cmp_col:
            cmp_frac = gk[cmp_col].fillna(80.0) / 100.0
            gk_score = (cmp_frac * 20.0)  # 0–20

        # Errors penalty
        err_col = pick_col(available_cols, 'Err')
        if err_col:
            penalty = (gk[err_col].fillna(0.0) * 2.5).clip(upper=5.0)
            gk_score = (gk_score - penalty)

        # Floor
        gk_score = gk_score.clip(lower=5.0)
        df.loc[gk_mask, 'Rebalanced_Score'] = gk_score
        print(f"  GK score range: {gk_score.min():.1f} – {gk_score.max():.1f}")

    # ---------------- Forwards ----------------
    fw_mask = df['Position_Group'].eq('Forward')
    if fw_mask.any():
        print(f"\nRebalancing Forwards ({int(fw_mask.sum())} rows)...")
        fw = df.loc[fw_mask].copy()
        fw['Min_adj'] = fw[MIN_COL].clip(lower=10)

        fw_score = pd.Series(0.0, index=fw.index)

        gls_col = pick_col(available_cols, ' Gls', 'Gls')
        if gls_col:
            gls90 = fw[gls_col] / fw['Min_adj'] * 90
            fw_score = fw_score.add((gls90 * 10).clip(upper=10))

        ast_col = pick_col(available_cols, ' Ast', 'Ast')
        if ast_col:
            ast90 = fw[ast_col] / fw['Min_adj'] * 90
            fw_score = fw_score.add((ast90 * 8).clip(upper=8))

        sh_col = pick_col(available_cols, ' Sh', 'Sh')
        if sh_col:
            sh90 = fw[sh_col] / fw['Min_adj'] * 90
            fw_score = fw_score.add((sh90 * 0.5).clip(upper=5))

        sot_col = pick_col(available_cols, ' SoT', 'SoT')
        if sot_col:
            sot90 = fw[sot_col] / fw['Min_adj'] * 90
            fw_score = fw_score.add((sot90 * 1.0).clip(upper=6))

        xg_col = pick_col(available_cols, 'Expected xG', 'xG')
        if xg_col:
            xg90 = fw[xg_col] / fw['Min_adj'] * 90
            fw_score = fw_score.add((xg90 * 5).clip(upper=5))

        # Minutes bonus
        fw_score = fw_score.add((fw[MIN_COL] / 90.0 * 0.1).clip(upper=3.0))

        df.loc[fw_mask, 'Rebalanced_Score'] = fw_score
        print(f"  FW score range: {fw_score.min():.1f} – {fw_score.max():.1f}")

    # ---------------- Midfielders ----------------
    mid_mask = df['Position_Group'].eq('Midfield')
    if mid_mask.any():
        print(f"\nRebalancing Midfielders ({int(mid_mask.sum())} rows)...")
        mid = df.loc[mid_mask].copy()
        mid['Min_adj'] = mid[MIN_COL].clip(lower=10)

        mid_score = pd.Series(0.0, index=mid.index)

        ast_col = pick_col(available_cols, ' Ast', 'Ast')
        if ast_col:
            ast90 = mid[ast_col] / mid['Min_adj'] * 90
            mid_score = mid_score.add((ast90 * 6).clip(upper=6))

        kp_col = pick_col(available_cols, 'KP')
        if kp_col:
            kp90 = mid[kp_col] / mid['Min_adj'] * 90
            mid_score = mid_score.add((kp90 * 1.5).clip(upper=5))

        prgp_col = pick_col(available_cols, 'Passes PrgP')
        if prgp_col:
            prgp90 = mid[prgp_col] / mid['Min_adj'] * 90
            mid_score = mid_score.add((prgp90 * 0.3).clip(upper=4))

        tkl_col = pick_col(available_cols, ' Tkl', 'Tkl')
        if tkl_col:
            tkl90 = mid[tkl_col] / mid['Min_adj'] * 90
            mid_score = mid_score.add((tkl90 * 1.0).clip(upper=3))

        passcmp_col = pick_col(available_cols, 'Passes Cmp%')
        if passcmp_col:
            acc = mid[passcmp_col].fillna(85.0)
            acc_score = ((acc - 80.0) / 20.0 * 4.0).clip(lower=0.0, upper=4.0)
            mid_score = mid_score.add(acc_score)

        mid_score = mid_score.add((mid[MIN_COL] / 90.0 * 0.1).clip(upper=4.0))

        df.loc[mid_mask, 'Rebalanced_Score'] = mid_score
        print(f"  MID score range: {mid_score.min():.1f} – {mid_score.max():.1f}")

    # ---------------- Defenders ----------------
    def_mask = df['Position_Group'].eq('Defense')
    if def_mask.any():
        print(f"\nRebalancing Defenders ({int(def_mask.sum())} rows)...")
        de = df.loc[def_mask].copy()
        de['Min_adj'] = de[MIN_COL].clip(lower=10)

        de_score = pd.Series(0.0, index=de.index)

        tklw_col = pick_col(available_cols, 'Tackles TklW')
        if tklw_col:
            tklw90 = de[tklw_col] / de['Min_adj'] * 90
            de_score = de_score.add((tklw90 * 2.0).clip(upper=6.0))
        else:
            tkl_col = pick_col(available_cols, ' Tkl', 'Tkl')
            if tkl_col:
                tkl90 = de[tkl_col] / de['Min_adj'] * 90
                de_score = de_score.add((tkl90 * 1.5).clip(upper=6.0))

        int_col = pick_col(available_cols, 'Int', ' Int')
        if int_col:
            int90 = de[int_col] / de['Min_adj'] * 90
            de_score = de_score.add((int90 * 2.0).clip(upper=6.0))

        blk_col = pick_col(available_cols, 'Blocks', ' Blocks')
        if blk_col:
            blk90 = de[blk_col] / de['Min_adj'] * 90
            de_score = de_score.add((blk90 * 3.0).clip(upper=6.0))

        clr_col = pick_col(available_cols, 'Clr')
        if clr_col:
            clr90 = de[clr_col] / de['Min_adj'] * 90
            de_score = de_score.add((clr90 * 0.5).clip(upper=4.0))

        de_score = de_score.add((de[MIN_COL] / 90.0 * 0.1).clip(upper=4.0))

        df.loc[def_mask, 'Rebalanced_Score'] = de_score
        print(f"  DEF score range: {de_score.min():.1f} – {de_score.max():.1f}")

    return df

# Apply fixed rebalanced scoring
df = calculate_rebalanced_scores_fixed(df)

# ==========================================
# VALIDATION
# ==========================================
print("\n✅ VALIDATION OF FIXED SYSTEM")

# Score distribution by position
print("\nScore Distribution by Position:")
pos_stats = (df.groupby('Position_Group')['Rebalanced_Score']
               .agg(['mean','median','std','min','max','count'])
               .round(2))
print(pos_stats)

# Top performers
print("\n🏆 TOP 15 PERFORMERS (All Positions):")
cols_to_show = ['Player', 'Position_Group', 'Rebalanced_Score', MIN_COL]
if 'Season' in df.columns:
    cols_to_show.append('Season')
print(df.nlargest(15, 'Rebalanced_Score')[cols_to_show].to_string(index=False))

# ==========================================
# SAVE REBALANCED DATASET
# ==========================================
print("\n💾 SAVING REBALANCED DATASET...")
output_path = Path('/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv')

# FIX: create the folder if it doesn't exist
output_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Rebalanced dataset saved to: {output_path}")
print(f"📊 Dataset contains {len(df)} rows and {len(df.columns)} columns")
print("🎯 New column 'Rebalanced_Score' added successfully")

# Quick verification
sample_cols = ['Player', 'Position_Group', 'Rebalanced_Score', MIN_COL]
if 'Season' in df.columns:
    sample_cols.append('Season')
print("\n📋 Sample of saved data:")
print(df[sample_cols].head(10).to_string(index=False))

print("\n✅ FIXED REBALANCED SCORING COMPLETE!")

=== FIXED PERFORMANCE SCORING SYSTEM ===
Dataset shape: (7140, 78)
Available columns: ['Date', 'Competition', 'Opponent', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', ' Gls', ' Ast', ' PK', ' PKatt', ' Sh', ' SoT', ' CrdY', ' CrdR', ' Int', 'Match URL', 'Season', ' Touches', ' Tkl', ' Blocks', 'Expected xG', 'Expected npxG', 'Expected xAG', 'SCA', 'GCA', 'Passes Cmp', 'Passes Att', 'Passes Cmp%', 'Passes PrgP', 'Carries Carries', 'Carries PrgC', 'Take-Ons Att', 'Take-Ons Succ', 'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist', 'Total PrgDist', 'Short Cmp', 'Short Att', 'Short Cmp%', 'Medium Cmp', 'Medium Att', 'Medium Cmp%', 'Long Cmp', 'Long Att', 'Long Cmp%', 'Ast', 'xAG', 'xA', 'KP', '3-Jan', 'PPA', 'CrsPA', 'PrgP', 'SCA SCA', 'SCA GCA', ' 

### What the scoring code did

- Built `Position_Group` from `Pos` and detected minutes column.
- Computed a position‑aware `Rebalanced_Score`:
  - Forwards: goals, assists, shots, shots on target, xG, minutes bonus.
  - Midfielders: assists, key passes, progressive passes, tackles, pass accuracy, minutes bonus.
  - Defenders: tackles won (or tackles), interceptions, blocks, clearances, minutes bonus.
  - Goalkeepers: distribution accuracy with penalties for errors and a floor.
- Printed distribution stats and top performers, then saved the CSV to:
  - `Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv`.
- Quick sample was displayed to verify the new `Rebalanced_Score` column.
